<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li></ul></li><li><span><a href="#LDA-based" data-toc-modified-id="LDA-based-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LDA based</a></span><ul class="toc-item"><li><span><a href="#tag-parser" data-toc-modified-id="tag-parser-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>tag parser</a></span><ul class="toc-item"><li><span><a href="#output-topic-distribution-and-then-clustering" data-toc-modified-id="output-topic-distribution-and-then-clustering-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>output topic distribution and then clustering</a></span></li></ul></li><li><span><a href="#Kmeans-clustering-based-on-LDA-topic-distribution-representation" data-toc-modified-id="Kmeans-clustering-based-on-LDA-topic-distribution-representation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Kmeans clustering based on LDA topic distribution representation</a></span></li></ul></li></ul></div>

In [1]:
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn import mixture
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer, cosine_distance
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from nltk.corpus import stopwords

In [2]:
seeds = 1234

In [3]:
train = pd.read_json('../data/structured_train.json')
test = pd.read_json('../data/structured_test.json')

In [4]:
# train = train.groupby('label').sample(50, random_state=seeds)
# test = test.groupby('label').sample(50, random_state=seeds)

In [5]:
select_cols = ["global_index", "doc_path", "label", 
               "reply", "reference_one", "reference_two","tag_reply", "tag_reference_one", "tag_reference_two", 
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"
               ]
print("\nmay use cols: \n", select_cols)
train = train[select_cols]
test = test[select_cols]


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report

## modeling

# LDA based 

## tag parser

In [6]:
from clustering_utils import *
# count_vectorizer, dimension_reduction


In [7]:
num_topics = 100

# train_text = train['tag_reply']
# train_text = train['tag_reply']+ ' ' + train['tag_reference_one']
train_text = train['reply'] + ' ' + train['reference_one']
train_label = train['label']


# test_text  = test['tag_reply'] 
# test_text  = test['tag_reply'] + ' ' + test['tag_reference_one']
test_text  = test['reply'] + ' ' + test['reference_one']
test_label = test['label']

lda, voc = fit_topic_model(train_text, num_topics=num_topics, save_name='gensim_lda_model')

In [8]:
# vis_lda = visualize_LDA_model(train_text, voc, lda)

In [14]:
# vis_lda
train_topic_distribution

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.014466,0.001682,0.001340,0.002473,0.000000,0.000000,0.000000,0.003366,0.000000,0.000000,...,0.000000,0.004425,0.083092,0.000000,0.000000,0.000000,0.000643,0.000000,0.000000,0.000000
1,0.012570,0.000000,0.000000,0.000776,0.000214,0.000790,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000000
2,0.011188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.487392,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,...,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.288031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11078,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,...,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400
11079,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.015601,0.000152,...,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152
11080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.105671,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11081,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526


In [16]:
train_topic_distribution

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.014466,0.001682,0.001340,0.002473,0.000000,0.000000,0.000000,0.003366,0.000000,0.000000,...,0.000000,0.004425,0.083092,0.000000,0.000000,0.000000,0.000643,0.000000,0.000000,0.000000
1,0.012570,0.000000,0.000000,0.000776,0.000214,0.000790,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000000
2,0.011188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.487392,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,...,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.000154,0.288031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11078,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,...,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400,0.000400
11079,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.015601,0.000152,...,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152,0.000152
11080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.105671,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11081,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526


In [ ]:

def link_group_to_label(train_label, train_pred):
    """with majority vote rule"""
    # Maping clusters into labels
    df = pd.DataFrame(list(zip(train_label, train_pred)), columns=['actual_class', 'cluster'])
    confusion = pd.crosstab(index=df.cluster, columns=df.actual_class)
    group_to_label = confusion.idxmax(axis=1)

    print("Group to label mapping: ")
    for idx, t in enumerate(group_to_label):
        print("Group {} <-> label {}".format(idx, t))
    print("\n")
    return group_to_label

In [21]:
confusion

actual_class,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
cluster,,,,,,,,,,,,,,,,,,,,
0,4,45,34,24,41,29,94,26,10,9,2,62,48,31,34,2,9,5,5,4
1,1,6,4,6,2,2,3,1,0,0,0,0,1,0,0,0,0,7,0,1
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,4,5,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0
5,0,0,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
91,1,6,18,19,8,4,5,3,3,4,0,0,2,0,0,1,1,0,1,0
92,12,16,6,17,3,6,17,2,5,12,6,3,12,4,7,5,1,7,1,5


In [18]:
train_pred = train_pred_group

In [19]:
df = pd.DataFrame(list(zip(train_label, train_pred)), columns=['actual_class', 'cluster'])
confusion = pd.crosstab(index=df.cluster, columns=df.actual_class)
group_to_label = confusion.idxmax(axis=1)

In [20]:
group_to_label

cluster
0                 misc.forsale
1        talk.politics.mideast
2                comp.graphics
3     comp.sys.ibm.pc.hardware
5      comp.os.ms-windows.misc
                ...           
90               comp.graphics
91    comp.sys.ibm.pc.hardware
92    comp.sys.ibm.pc.hardware
96     comp.os.ms-windows.misc
97    comp.sys.ibm.pc.hardware
Length: 68, dtype: object

In [10]:
train_pred_group, train_topic_distribution = pred_topic_model(lda, train_text, vocabulary=voc)
group_to_label = link_group_to_label(train_label, train_pred_group)

test_pred_group, test_topic_distribution = pred_topic_model(lda, test_text, vocabulary=voc)
test_pred = test_pred_group.apply(lambda group: group_to_label[group])

Series([], dtype: float64)
Group to label mapping: 
Group 0 <-> label misc.forsale
Group 1 <-> label talk.politics.mideast
Group 2 <-> label comp.graphics
Group 3 <-> label comp.sys.ibm.pc.hardware
Group 4 <-> label comp.os.ms-windows.misc
Group 5 <-> label comp.os.ms-windows.misc
Group 6 <-> label comp.graphics
Group 7 <-> label comp.os.ms-windows.misc
Group 8 <-> label alt.atheism
Group 9 <-> label comp.os.ms-windows.misc
Group 10 <-> label comp.graphics
Group 11 <-> label comp.graphics
Group 12 <-> label comp.sys.ibm.pc.hardware
Group 13 <-> label comp.sys.ibm.pc.hardware
Group 14 <-> label comp.sys.mac.hardware
Group 15 <-> label comp.windows.x
Group 16 <-> label comp.os.ms-windows.misc
Group 17 <-> label talk.politics.mideast
Group 18 <-> label alt.atheism
Group 19 <-> label comp.sys.mac.hardware
Group 20 <-> label comp.windows.x
Group 21 <-> label comp.graphics
Group 22 <-> label rec.sport.baseball
Group 23 <-> label soc.religion.christian
Group 24 <-> label comp.sys.ibm.pc.hardw

KeyError: 94

In [ ]:
print(metrics.classification_report(y_true = test_label, y_pred=test_pred))

### output topic distribution and then clustering

In [ ]:
# Construct the k-means clusters
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=10, random_state=100).fit_predict(lda_output)

# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)

# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]

# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))

# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))

# 
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )

## Kmeans clustering based on LDA topic distribution representation